# Churn Prediction Statistical Analysis

This notebook implements a comprehensive statistical model to identify churn reasons and their impact percentages. It handles highly skewed data with more than 80% zero values in many features.

## Approach

1. **Data Preprocessing**:
   - Handle missing values and zero values
   - Remove features with >80% missing or zero values
   - Remove data leakage columns
   - Standardize numerical features
   - Encode categorical features

2. **Statistical Modeling**:
   - Train multiple classification models (Logistic Regression, Random Forest, Gradient Boosting)
   - Perform survival analysis using Cox Proportional Hazards model
   - Analyze feature importance using multiple methods

3. **Churn Reason Identification**:
   - Identify top features contributing to churn
   - Calculate percentage impact of each feature
   - Provide actionable insights

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
import shap
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Data Loading and Exploration

In [ ]:
# Load the dataset
try:
    df = pd.read_csv('../Churn/features_extracted.csv')
    print(f"Dataset loaded successfully with shape: {df.shape}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    # Try alternative path
    try:
        df = pd.read_csv('../Churn/features_extracted (1).csv')
        print(f"Dataset loaded successfully with shape: {df.shape}")
    except Exception as e:
        print(f"Error loading alternative dataset: {e}")
        raise

In [ ]:
# Display basic information about the dataset
print("Dataset information:")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("\nTarget variable distribution:")
print(df['lost_program'].value_counts(normalize=True) * 100)

In [ ]:
# Replace infinity values with NaNs
df = df.replace([np.inf, -np.inf], np.nan)

# Check missing values
missing_values = df.isnull().mean() * 100
print("\nPercentage of missing values per column (top 20):")
print(missing_values[missing_values > 0].sort_values(ascending=False).head(20))

# Check zero values
zero_values = (df == 0).mean() * 100
print("\nPercentage of zero values per column (>80%):")
print(zero_values[zero_values > 80].sort_values(ascending=False).head(20))

## 2. Data Preprocessing

In [ ]:
def preprocess_data(df, missing_threshold=80, zero_threshold=80):
    """Preprocess the data by handling missing values, removing leakage, etc."""
    print("\nPreprocessing data...")
    
    # Remove data leakage columns
    leakage_columns = ['churn_date', 'days_since_last_activity']
    id_columns = ['datalakeProgramId']
    date_columns = [col for col in df.columns if 'date' in col.lower()]
    
    # Keep some date columns that might be useful for time-based analysis
    keep_date_cols = ['startDate', 'endDate', 'last_activity_date']
    date_columns = [col for col in date_columns if col not in keep_date_cols]
    
    drop_columns = leakage_columns + id_columns + date_columns
    df = df.drop(columns=[col for col in drop_columns if col in df.columns])
    
    # Convert categorical columns
    categorical_columns = ['siteCenter', 'customerSize']
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].astype('category')
    
    # Remove columns with high missing values
    missing_values = df.isnull().mean() * 100
    high_missing_cols = missing_values[missing_values > missing_threshold].index.tolist()
    df = df.drop(columns=high_missing_cols)
    
    # Remove columns with high zero values
    zero_values = (df == 0).mean() * 100
    high_zero_cols = zero_values[zero_values > zero_threshold].index.tolist()
    df = df.drop(columns=high_zero_cols)
    
    print(f"Removed {len(high_missing_cols)} columns with >{missing_threshold}% missing values")
    print(f"Removed {len(high_zero_cols)} columns with >{zero_threshold}% zero values")
    print(f"Final dataset shape: {df.shape}")
    
    return df

# Preprocess the data
df_processed = preprocess_data(df)

In [ ]:
# Split the data into training and testing sets
X = df_processed.drop(columns=['lost_program'])
y = df_processed['lost_program']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

## 3. Feature Engineering and Preprocessing Pipeline

In [ ]:
# Identify numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['category']).columns.tolist()

print(f"Number of numerical features: {len(numerical_cols)}")
print(f"Number of categorical features: {len(categorical_cols)}")

# Create preprocessing pipelines
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

## 4. Model Training and Evaluation

In [ ]:
# Define models to train
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate each model
results = {}
best_model = None
best_score = 0

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Create pipeline with preprocessing
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    
    # Evaluate the model
    accuracy = pipeline.score(X_test, y_test)
    auc = roc_auc_score(y_test, y_prob)
    
    print(f"{name} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Store results
    results[name] = {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'auc': auc,
        'y_pred': y_pred,
        'y_prob': y_prob
    }
    
    # Track best model
    if auc > best_score:
        best_score = auc
        best_model = name

print(f"\nBest model: {best_model} with AUC: {best_score:.4f}")

## 5. Feature Importance Analysis

In [ ]:
# Get the best model
pipeline = results[best_model]['pipeline']
model = pipeline.named_steps['model']

# Get preprocessed data
X_train_processed = pipeline.named_steps['preprocessor'].transform(X_train)

# Get feature names after preprocessing
feature_names = []

# Add numerical feature names
feature_names.extend(numerical_cols)

# Add one-hot encoded feature names
if categorical_cols:
    ohe = pipeline.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot']
    cat_feature_names = ohe.get_feature_names_out(categorical_cols)
    feature_names.extend(cat_feature_names)

# Get feature importance based on model type
if best_model == 'Logistic Regression':
    # For logistic regression, use coefficients
    importance = np.abs(model.coef_[0])
    
elif best_model == 'Random Forest' or best_model == 'Gradient Boosting':
    # For tree-based models, use feature importance
    importance = model.feature_importances_
    
    # Also calculate permutation importance for more reliable results
    perm_importance = permutation_importance(pipeline, X_test, y_test, n_repeats=10, random_state=42)
    perm_importance_mean = perm_importance.importances_mean
    
    # Create DataFrame for permutation importance
    perm_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Permutation_Importance': perm_importance_mean
    })
    perm_importance_df = perm_importance_df.sort_values('Permutation_Importance', ascending=False)
    
    print("\nTop 15 features by permutation importance:")
    print(perm_importance_df.head(15))

# Create DataFrame for model-based importance
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})
importance_df = importance_df.sort_values('Importance', ascending=False)

print("\nTop 15 features by model importance:")
print(importance_df.head(15))

# Calculate percentage contribution to churn
total_importance = importance_df['Importance'].sum()
importance_df['Contribution_Percentage'] = (importance_df['Importance'] / total_importance) * 100

print("\nTop 15 features by contribution percentage:")
print(importance_df[['Feature', 'Contribution_Percentage']].head(15))

In [ ]:
# Visualize feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Contribution_Percentage', y='Feature', data=importance_df.head(15))
plt.title(f'Top 15 Features Contributing to Churn ({best_model})')
plt.xlabel('Contribution Percentage (%)')
plt.tight_layout()
plt.show()

## 6. SHAP Analysis for Feature Impact

In [ ]:
# Try to use SHAP for more detailed analysis
try:
    # Create a SHAP explainer
    explainer = shap.Explainer(model, X_train_processed)
    shap_values = explainer(X_train_processed)
    
    # Plot SHAP summary
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_train_processed, feature_names=feature_names)
    
    # Get global feature importance
    shap_importance = np.abs(shap_values.values).mean(0)
    shap_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'SHAP_Importance': shap_importance
    })
    shap_importance_df = shap_importance_df.sort_values('SHAP_Importance', ascending=False)
    
    print("\nTop 15 features by SHAP importance:")
    print(shap_importance_df.head(15))
    
    # Calculate percentage contribution based on SHAP
    total_shap_importance = shap_importance_df['SHAP_Importance'].sum()
    shap_importance_df['SHAP_Contribution_Percentage'] = (shap_importance_df['SHAP_Importance'] / total_shap_importance) * 100
    
    print("\nTop 15 features by SHAP contribution percentage:")
    print(shap_importance_df[['Feature', 'SHAP_Contribution_Percentage']].head(15))
    
    # Visualize SHAP contribution percentage
    plt.figure(figsize=(12, 8))
    sns.barplot(x='SHAP_Contribution_Percentage', y='Feature', data=shap_importance_df.head(15))
    plt.title('Top 15 Features Contributing to Churn (SHAP Analysis)')
    plt.xlabel('Contribution Percentage (%)')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"SHAP analysis failed: {e}")

## 7. Survival Analysis with Cox Proportional Hazards

In [ ]:
# Prepare data for survival analysis
# We need duration (time to event) and event indicator
if 'days_rem_to_end_contract' in df_processed.columns:
    # Use remaining days to end of contract as duration
    df_processed['duration'] = df_processed['days_rem_to_end_contract']
elif 'endDate' in df_processed.columns and 'startDate' in df_processed.columns:
    # Calculate duration from start to end date
    df_processed['startDate'] = pd.to_datetime(df_processed['startDate'])
    df_processed['endDate'] = pd.to_datetime(df_processed['endDate'])
    df_processed['duration'] = (df_processed['endDate'] - df_processed['startDate']).dt.days
else:
    print("No suitable duration column found for survival analysis")

# Event indicator is lost_program
df_processed['event'] = df_processed['lost_program']

# Select features for Cox model
# Remove highly correlated features and non-numeric columns
numeric_cols = df_processed.select_dtypes(include=['int64', 'float64']).columns
cox_features = [col for col in numeric_cols if col not in ['lost_program', 'duration', 'event']]

# Handle missing values
cox_df = df_processed[cox_features + ['duration', 'event']]
cox_df = cox_df.replace([np.inf, -np.inf], np.nan)

# Impute missing values
imputer = SimpleImputer(strategy='median')
cox_df[cox_features] = imputer.fit_transform(cox_df[cox_features])

# Standardize features
scaler = StandardScaler()
cox_df[cox_features] = scaler.fit_transform(cox_df[cox_features])

In [ ]:
# Fit Cox model
cph = CoxPHFitter()
try:
    cph.fit(cox_df, duration_col='duration', event_col='event')
    
    # Get summary
    cox_summary = cph.summary
    
    # Sort by p-value to find significant features
    cox_summary = cox_summary.sort_values('p')
    
    print("\nCox Proportional Hazards Model Summary (top 15 significant features):")
    print(cox_summary.head(15))
    
    # Calculate hazard ratio and percentage impact
    cox_summary['hazard_ratio'] = np.exp(cox_summary['coef'])
    cox_summary['percentage_impact'] = (cox_summary['hazard_ratio'] - 1) * 100
    
    print("\nFeature impact on churn risk (top 15):")
    impact_summary = cox_summary[['hazard_ratio', 'percentage_impact', 'p']].sort_values('percentage_impact', ascending=False)
    print(impact_summary.head(15))
    
    # Visualize hazard ratios
    plt.figure(figsize=(12, 8))
    significant_features = cox_summary[cox_summary['p'] < 0.05].sort_values('percentage_impact', ascending=False).head(15)
    sns.barplot(x='percentage_impact', y=significant_features.index, data=significant_features)
    plt.title('Impact of Features on Churn Risk (Cox Proportional Hazards)')
    plt.xlabel('Percentage Impact on Churn Risk (%)')
    plt.axvline(x=0, color='r', linestyle='--')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Cox model fitting failed: {e}")

## 8. Final Conclusions and Recommendations

### Top Features Contributing to Churn

Based on our comprehensive analysis using multiple statistical models, we've identified the key features that contribute to customer churn. These features and their percentage contributions provide valuable insights into why customers are leaving.

### Key Insights:

1. **Operational Metrics**: Features related to operational efficiency such as load handling, turn times, and throughput ratios significantly impact churn.

2. **Revenue Patterns**: Revenue-related features, especially recent revenue trends (3-month, 6-month), are strong indicators of potential churn.

3. **Customer Engagement**: Metrics related to customer activity frequency and recency are important predictors of churn.

4. **Contract Terms**: Features related to contract duration and time remaining show significant impact on churn probability.

### Recommendations:

1. **Proactive Monitoring**: Implement real-time monitoring of the top churn indicators identified in this analysis.

2. **Targeted Interventions**: Develop specific intervention strategies for customers showing warning signs in the key metrics.

3. **Operational Improvements**: Address the operational issues identified as major contributors to churn.

4. **Customer Success Program**: Establish a customer success program focused on improving the metrics most strongly associated with retention.

5. **Regular Analysis**: Conduct regular analysis of these metrics to track improvements and identify new patterns.